# Compare excel vs csv to ensure no records were formated wrong

In [ ]:
import pandas as pd
import numpy as np
from typing import Dict, List, Tuple, Any

def compare_column_text(excel_file: str, csv_file: str, excel_column: str, csv_column: str,
                       excel_sheet: str = None) -> Dict[str, Any]:
    """
    Compare text content of columns between Excel and CSV files.
    
    Args:
        excel_file: Path to Excel file
        csv_file: Path to CSV file  
        excel_column: Name of column in Excel file
        csv_column: Name of column in CSV file
        excel_sheet: Sheet name for Excel (None for first sheet)
    
    Returns:
        Dictionary with comparison results
    """
    
    # Read files
    try:
        if excel_sheet:
            df_excel = pd.read_excel(excel_file, sheet_name=excel_sheet)
        else:
            df_excel = pd.read_excel(excel_file)
        
        df_csv = pd.read_csv(csv_file)
        
    except Exception as e:
        return {'error': f"Error reading files: {str(e)}"}
    
    # Check if columns exist in respective files
    if excel_column not in df_excel.columns:
        return {'error': f"Column '{excel_column}' not found in Excel file. Available: {list(df_excel.columns)}"}
    
    if csv_column not in df_csv.columns:
        return {'error': f"Column '{csv_column}' not found in CSV file. Available: {list(df_csv.columns)}"}
    
    # Extract columns and convert to string for text comparison
    excel_col = df_excel[excel_column].astype(str)
    csv_col = df_csv[csv_column].astype(str)
    
    # Basic statistics
    results = {
        'excel_rows': len(excel_col),
        'csv_rows': len(csv_col),
        'rows_match': len(excel_col) == len(csv_col)
    }
    
    # Compare content
    if results['rows_match']:
        # Exact matches
        exact_matches = (excel_col == csv_col).sum()
        results['exact_matches'] = exact_matches
        results['exact_match_rate'] = exact_matches / len(excel_col)
        
        # Find differences
        differences = []
        for i, (excel_val, csv_val) in enumerate(zip(excel_col, csv_col)):
            if excel_val != csv_val:
                differences.append({
                    'row': i,
                    'excel_value': excel_val,
                    'csv_value': csv_val
                })
        
        results['differences'] = differences[:10]  # First 10 differences
        results['total_differences'] = len(differences)
        
    else:
        # Different lengths - compare overlapping portion
        min_len = min(len(excel_col), len(csv_col))
        excel_subset = excel_col[:min_len]
        csv_subset = csv_col[:min_len]
        
        exact_matches = (excel_subset == csv_subset).sum()
        results['exact_matches_overlap'] = exact_matches
        results['exact_match_rate_overlap'] = exact_matches / min_len if min_len > 0 else 0
    
    # Sample values for inspection
    results['excel_sample'] = excel_col.head(5).tolist()
    results['csv_sample'] = csv_col.head(5).tolist()
    
    # Check for common issues
    issues = []
    
    # Check for NaN handling differences
    excel_nulls = df_excel[excel_column].isnull().sum()
    csv_nulls = df_csv[csv_column].isnull().sum()
    if excel_nulls != csv_nulls:
        issues.append(f"Different null counts: Excel={excel_nulls}, CSV={csv_nulls}")
    
    # Check for whitespace issues
    excel_stripped = df_excel[excel_column].astype(str).str.strip()
    csv_stripped = df_csv[csv_column].astype(str).str.strip()
    if results['rows_match']:
        stripped_matches = (excel_stripped == csv_stripped).sum()
        if stripped_matches > exact_matches:
            issues.append("Whitespace differences detected")
    
    # Check for encoding issues (common characters that might differ)
    encoding_chars = ['"', '"', ''', ''', '–', '—']
    for char in encoding_chars:
        excel_has = excel_col.str.contains(char, na=False).any()
        csv_has = csv_col.str.contains(char, na=False).any()
        if excel_has != csv_has:
            issues.append(f"Encoding difference detected with character: {char}")
    
    results['potential_issues'] = issues
    
    return results

def print_comparison_report(comparison_results: Dict[str, Any], excel_column: str, csv_column: str):
    """Print a formatted report of the comparison results."""
    
    if 'error' in comparison_results:
        print(f"❌ Error: {comparison_results['error']}")
        return
    
    results = comparison_results
    
    print(f"📊 Column Comparison Report")
    print(f"Excel: '{excel_column}' vs CSV: '{csv_column}'")
    print("=" * 60)
    
    print(f"📈 Row Counts:")
    print(f"  Excel: {results['excel_rows']:,} rows")
    print(f"  CSV: {results['csv_rows']:,} rows")
    print(f"  Lengths match: {'✅' if results['rows_match'] else '❌'}")
    
    if results['rows_match']:
        match_rate = results['exact_match_rate'] * 100
        print(f"\n🎯 Content Matching:")
        print(f"  Exact matches: {results['exact_matches']:,} / {results['excel_rows']:,}")
        print(f"  Match rate: {match_rate:.2f}%")
        
        if results['total_differences'] > 0:
            print(f"\n❌ Differences found: {results['total_differences']:,}")
            print("First few differences:")
            for diff in results['differences'][:3]:
                print(f"  Row {diff['row']}: Excel='{diff['excel_value']}' | CSV='{diff['csv_value']}'")
    else:
        if 'exact_match_rate_overlap' in results:
            overlap_rate = results['exact_match_rate_overlap'] * 100
            print(f"  Overlap match rate: {overlap_rate:.2f}%")
    
    print(f"\n🔍 Sample Values:")
    print("Excel:", results['excel_sample'])
    print("CSV:  ", results['csv_sample'])
    
    if results['potential_issues']:
        print(f"\n⚠️  Potential Issues:")
        for issue in results['potential_issues']:
            print(f"  • {issue}")
    else:
        print(f"\n✅ No common issues detected")
    
    print("\n" + "="*60 + "\n")

# McCray comparison script
def compare_mccray_files():
    """Compare specific columns from McCray Excel and CSV files"""
    
    excel_path = r"/home/steven-dindl/Documents/AspireAI_LLM_Project/data/mccray/changed_data/McCray+.xlsx"
    csv_path = r"/home/steven-dindl/Documents/AspireAI_LLM_Project/data/mccray/changed_data/McCray+.csv"
    
    # Columns to compare (assuming same names in both files)
    # columns_to_compare = ['Extent', 'CONTENTdm number', 'Reference URL']
    columns_to_compare = ['Original Transcript']
    
    print("🔍 McCray Files Column Comparison")
    print("=" * 80)
    print(f"Excel file: {excel_path}")
    print(f"CSV file: {csv_path}")
    print("=" * 80)
    
    for column in columns_to_compare:
        print(f"\n📋 Comparing column: '{column}'")
        
        results = compare_column_text(
            excel_file=excel_path,
            csv_file=csv_path,
            excel_column=column,
            csv_column=column
        )
        
        print_comparison_report(results, column, column)


In [9]:
# # compare 'Extent, 'CONTENTdm number', 'Reference URL'
# excel_path = r"/home/steven-dindl/Documents/AspireAI_LLM_Project/data/mccray/changed_data/McCray+.xlsx"
# csv_path = r"/home/steven-dindl/Documents/AspireAI_LLM_Project/data/mccray/changed_data/McCray+.csv"

compare_mccray_files()

🔍 McCray Files Column Comparison
Excel file: /home/steven-dindl/Documents/AspireAI_LLM_Project/data/mccray/changed_data/McCray+.xlsx
CSV file: /home/steven-dindl/Documents/AspireAI_LLM_Project/data/mccray/changed_data/McCray+.csv

📋 Comparing column: 'Original Transcript'
📊 Column Comparison Report
Excel: 'Original Transcript' vs CSV: 'Original Transcript'
📈 Row Counts:
  Excel: 19,329 rows
  CSV: 19,329 rows
  Lengths match: ✅

🎯 Content Matching:
  Exact matches: 4,405 / 19,329
  Match rate: 22.79%

❌ Differences found: 14,924
First few differences:
  Row 0: Excel='-5491  AFRO-AMERICAN  NEWSBOY'S  APPLICATION  I hereby apply for membership in the AFRO Newsboys' Association of Carriers  and Street Salesmen, and if accepted, this application being properly signed by myself,  parents or guardian, shall be construed as ' | CSV='-5491  AFRO-AMERICAN  NEWSBOY'S  APPLICATION  I hereby apply for membership in the AFRO Newsboys' Association of Carriers  and Street Salesmen, and if accepted, t

In [ ]:
# ❌ Error: Column 'Transcript' not found in CSV file. Available: ['Title', 'Creator', 'Contributors', 'Date', 'Approximate Date', 'Source', 'Subject', 'Local Subject', 'S.C. County', 'Description', 'Extent', 'Digital Collection', 'Website', 'Contributing Institution', 'Rights', 'Time Period', 'Geographic Location', 'Language', 'Digitization Specifications', 'Date Digital', 'Type', 'Format', 'Media Type', 'Identifier', 'Note', 'Digital Assistant', 'OCLC number', 'Date created', 'Date modified', 'Reference URL', 'CONTENTdm number', 'CONTENTdm file name', 'CONTENTdm file path', 'Year', 'Original Transcript', 'Original Len', 'Special Pattern', 'General Pattern', 'Repeat Chars', 'Short/No Transcript', 'Quality', 'Issue Types', 'Detected Artifacts', 'Semi-clean Transcript']